# 介绍
本应用主要用于统计客服的转化收益。

统计准则是以客服支持后10天以内的计入客服转化收益。

# 开始
## 文件输入输出
共三个输入文件。分别是某段时间的：
- 工单报表.xlsx
- 学费报表.xlsx
- 会话记录报表.csv

x个输出文件：



默认均放置在'data'目录下

In [1]:
input_workorder_path = r'D:\yyfang\201206_income_statics\data\工单报表.xlsx'
input_session_records_path = r'D:\yyfang\201206_income_statics\data\会话记录报表.xlsx'
input_tuition_fee_path = r'‪D:\yyfang\201206_income_statics\data\学费报表.xlsx'
input_workorder_path = input_workorder_path.strip("‪u202a")
input_session_records_path = input_session_records_path.strip("‪u202a")
input_tuition_fee_path = input_tuition_fee_path.strip("‪u202a")

## 导入相关库

In [2]:
import numpy as np
import pandas as pd
import time
from matplotlib import pyplot as plt
%matplotlib inline
from pylab import *
mpl.rcParams['font.sans-serif'] = ['SimHei']

ModuleNotFoundError: No module named 'matplotlib'

第一步：把'会话记录报表'中有电话的筛出来，并存入*has_phone_number_data*页中

In [ ]:
session_records_data = pd.read_excel(input_session_records_path);
# session_records_data.head()

In [ ]:
has_phone_number_data = session_records_data.loc[session_records_data['电话'].notnull()].copy()
has_phone_number_data['电话'] = has_phone_number_data['电话'].apply(lambda x: '%s' % (str(int(x))))
has_phone_number_data = has_phone_number_data[['会话ID', '最后接待客服', '会话结束时间', '电话']]

In [ ]:
with pd.ExcelWriter(input_session_records_path, engine='openpyxl', mode='a') as writer:
    has_phone_number_data.to_excel(writer, sheet_name='有电话号')

第二步：把'会话记录报表'有效数据合并到'工单报表'的*merge*页

In [ ]:
workorder_data = pd.read_excel(input_workorder_path);

In [ ]:
valid_session_records_data = has_phone_number_data.rename(columns={'会话ID': '工单号', '最后接待客服': '创建人', '会话结束时间':'工单生成时间', '电话':'客户电话'})
# invalid_session_records_data.head()
workorder_data_copy = workorder_data.copy()
workorder_data_copy['客户电话'] = workorder_data_copy['客户电话'].apply(lambda x: '%s' % (str(int(x))))
merge_data = pd.concat([workorder_data_copy, valid_session_records_data], axis = 0)

In [ ]:
with pd.ExcelWriter(input_workorder_path, engine='openpyxl', mode='a') as writer:
    merge_data.to_excel(writer, sheet_name='1合并会话记录', index=False)

第三步：把'学费报表'中金额大于0的数据抽取出来，按金额降序，存入'工单报表'的*profitable_tuition_fee*页

In [3]:
tuition_fee_data = pd.read_excel(input_tuition_fee_path)
# tuition_fee_data.head()

D:\Users\liyanan\miniconda3\lib\site-packages\openpyxl\styles\stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [6]:
tuition_fee_data.head()

,订单/退款单编号


In [ ]:
profitable_tuition_fee_data = tuition_fee_data.loc[tuition_fee_data['收款/退款金额']>0].copy()
profitable_tuition_fee_data = profitable_tuition_fee_data[['手机号', '收款/退款金额', '收款/退款日期']]
profitable_tuition_fee_data = profitable_tuition_fee_data.sort_values(by=['收款/退款金额'], ascending=False)

In [ ]:
with pd.ExcelWriter(input_workorder_path, engine='openpyxl', mode='a') as writer:
    profitable_tuition_fee_data.to_excel(writer, sheet_name='2学费大于零', index=False)

第四步：把profitable_tuition_fee数据和merge数据通过手机号进行合并，并存入merge_money页

In [ ]:
valid_tuition_fee_data = profitable_tuition_fee_data.rename(columns={'手机号': '客户电话'})
valid_tuition_fee_data['客户电话'] = valid_tuition_fee_data['客户电话'].apply(lambda x: '%s' % (str(x)))
valid_tuition_fee_data = valid_tuition_fee_data.drop_duplicates('客户电话')
s = valid_tuition_fee_data['客户电话'].str.split('/').apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1) # to line up with df's index
s.name = '客户电话'
del valid_tuition_fee_data['客户电话']
valid_tuition_fee_data = valid_tuition_fee_data.join(s)

In [ ]:
merge_data['工单生成时间'] = pd.to_datetime(merge_data['工单生成时间'],format= '%Y-%m-%d %H:%M:%S' )   # 更改时间为统一格式
merge_data = merge_data.sort_values(by=['工单生成时间'], ascending=True)  # 同一个电话多次打入，去重，算最先接的那个人的
merge_data = merge_data.drop_duplicates('客户电话')
merge_money_data = pd.merge(left = merge_data, right = valid_tuition_fee_data, how = 'left', on = ['客户电话'])

In [ ]:
with pd.ExcelWriter(input_workorder_path, engine='openpyxl', mode='a') as writer:
    merge_money_data.to_excel(writer, sheet_name='3合并学费', index=False)

第五步：把merge_money数据中符合时间差的数据抽离出来，存入valid_income页

In [ ]:
valid_income_data = merge_money_data.loc[merge_money_data['收款/退款日期'].notnull()].copy()
valid_income_data['时间差'] = valid_income_data['收款/退款日期'] - valid_income_data['工单生成时间']
valid_income_data = valid_income_data.loc[(valid_income_data['时间差'] >= pd.Timedelta(0,'D')) &  (valid_income_data['时间差'] <= pd.Timedelta(10,'D'))].copy()
valid_income_data['时间差'] = valid_income_data['时间差'].astype(str)

In [ ]:
with pd.ExcelWriter(input_workorder_path, engine='openpyxl', mode='a') as writer:
    valid_income_data.to_excel(writer, sheet_name='4小于十天', index=False)

第六步：统计每个人的金额，存入personal_income页

In [ ]:
valid_income_data = valid_income_data.sort_values(by=['工单生成时间'], ascending=True)
valid_income_data = valid_income_data.drop_duplicates('客户电话')

In [ ]:
all_people = valid_income_data['创建人'].unique()
all_money = []
for p in all_people:
    m = valid_income_data.loc[valid_income_data['创建人']== p]['收款/退款金额'].sum()
    all_money.append(m)
people = pd.Series(data = all_people, name = '创建人')
money = pd.Series(data = all_money, name = '金额')
personal_money = pd.concat([people, money], axis=1)
personal_money = personal_money.sort_values(by=['金额'], ascending=False)

In [ ]:
with pd.ExcelWriter(input_workorder_path, engine='openpyxl', mode='a') as writer:
    personal_money.to_excel(writer, sheet_name='5个人统计', index=False)

In [ ]:
valid_income_data['创建人'].unique()

In [ ]:
merge_money_data.loc[merge_money_data['创建人']== '李曦然']

In [ ]:
merge_money_data